### Customer Risk Scoring Framework

In [4]:
import pandas as pd
from pathlib import Path

DATA_PATH = Path("../data/processed/telco_cleaned.csv")
df = pd.read_csv(DATA_PATH)

df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [5]:
# Month-to-month flag
df["risk_contract"] = (df["Contract"] == "Month-to-month").astype(int)

# Fiber optic flag
df["risk_internet"] = (df["InternetService"] == "Fiber optic").astype(int)

# Electronic check flag
df["risk_payment"] = (df["PaymentMethod"] == "Electronic check").astype(int)

# Early tenure flag (0-6 months)
df["risk_tenure"] = (df["tenure"] <= 6).astype(int)

# Senior citizen flag
df["risk_senior"] = (df["SeniorCitizen"] == 1).astype(int)

df[[
    "risk_contract",
    "risk_internet",
    "risk_payment",
    "risk_tenure",
    "risk_senior"
]].head()

,risk_contract,risk_internet,risk_payment,risk_tenure,risk_senior
0,1,0,1,1,0
1,0,0,0,0,0
2,1,0,0,1,0
3,0,0,0,0,0
4,1,1,1,1,0


###  Building Composite Risk Score

In [6]:
df["risk_score"] = (
    df["risk_contract"] +
    df["risk_internet"] +
    df["risk_payment"] +
    df["risk_tenure"] +
    df["risk_senior"]
)

df[[
    "risk_contract",
    "risk_internet",
    "risk_payment",
    "risk_tenure",
    "risk_senior",
    "risk_score"
]].head()

,risk_contract,risk_internet,risk_payment,risk_tenure,risk_senior,risk_score
0,1,0,1,1,0,3
1,0,0,0,0,0,0
2,1,0,0,1,0,2
3,0,0,0,0,0,0
4,1,1,1,1,0,4


In [7]:
df["risk_score"].max()

np.int64(5)

### Risk Segments

In [8]:
def assign_risk_segment(score):
    if score <= 1:
        return "Low"
    elif score <= 3:
        return "Medium"
    else:
        return "High"

df["risk_segment"] = df["risk_score"].apply(assign_risk_segment)

df[["risk_score", "risk_segment"]].head()

,risk_score,risk_segment
0,3,Medium
1,0,Low
2,2,Medium
3,0,Low
4,4,High


In [9]:
df["risk_segment"].value_counts()

risk_segment
Low       3313
Medium    2919
High       811
Name: count, dtype: int64

### Churn Rate by Risk Segment

In [10]:
risk_churn = pd.crosstab(
    df["risk_segment"],
    df["Churn"],
    normalize="index"
) * 100

risk_churn

Churn,No,Yes
risk_segment,,
High,32.182491,67.817509
Low,93.359493,6.640507
Medium,62.350120,37.649880


### Revenue Concentration by Risk Segment

In [11]:
risk_revenue_total = df.groupby("risk_segment")["MonthlyCharges"].sum()

risk_revenue_total

risk_segment
High       67133.25
Low       180417.20
Medium    208566.15
Name: MonthlyCharges, dtype: float64

In [12]:
risk_revenue_churn = (
    df[df["Churn"] == "Yes"]
    .groupby("risk_segment")["MonthlyCharges"]
    .sum()
)

risk_revenue_churn

risk_segment
High      45092.35
Low       14281.05
Medium    79757.45
Name: MonthlyCharges, dtype: float64

In [13]:
risk_revenue_risk_pct = (
    risk_revenue_churn / risk_revenue_total
) * 100

risk_revenue_risk_pct

risk_segment
High      67.168430
Low        7.915570
Medium    38.240841
Name: MonthlyCharges, dtype: float64

### Identifying Strongest Churn Signals

In [14]:
import sys
from pathlib import Path

sys.path.append(str(Path().resolve().parent))

In [15]:
from src.analysis import calculate_churn_rate

In [16]:
df["tenure_group"] = pd.cut(
    df["tenure"],
    bins=[-1, 6, 12, 24, 48, 72],
    labels=["0-6", "6-12", "12-24", "24-48", "48-72"]
)

In [17]:
df.columns

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn',
       'risk_contract', 'risk_internet', 'risk_payment', 'risk_tenure',
       'risk_senior', 'risk_score', 'risk_segment', 'tenure_group'],
      dtype='str')

In [18]:
signals = [
    "Contract",
    "InternetService",
    "PaymentMethod",
    "tenure_group",
    "SeniorCitizen"
]

for col in signals:
    print(f"\n=== {col} ===")
    display(calculate_churn_rate(df, col))


=== Contract ===


Churn,No,Yes
Contract,,
Month-to-month,57.290323,42.709677
One year,88.730482,11.269518
Two year,97.168142,2.831858



=== InternetService ===


Churn,No,Yes
InternetService,,
DSL,81.040892,18.959108
Fiber optic,58.107235,41.892765
No,92.595020,7.404980



=== PaymentMethod ===


Churn,No,Yes
PaymentMethod,,
Bank transfer (automatic),83.290155,16.709845
Credit card (automatic),84.756899,15.243101
Electronic check,54.714588,45.285412
Mailed check,80.893300,19.106700



=== tenure_group ===


Churn,No,Yes
tenure_group,,
0-6,47.062795,52.937205
6-12,64.113475,35.886525
12-24,71.289062,28.710938
24-48,79.611041,20.388959
48-72,90.486824,9.513176



=== SeniorCitizen ===


Churn,No,Yes
SeniorCitizen,,
0,76.393832,23.606168
1,58.318739,41.681261


### Financial Impact by Risk Signal

In [19]:
signals_conditions = {
    "Month-to-month": df["Contract"] == "Month-to-month",
    "Fiber optic": df["InternetService"] == "Fiber optic",
    "Electronic check": df["PaymentMethod"] == "Electronic check",
    "Tenure 0-6": df["tenure"] <= 6,
    "Senior Citizen": df["SeniorCitizen"] == 1
}

signal_results = []

for name, condition in signals_conditions.items():
    segment = df[condition]
    
    total_revenue = segment["MonthlyCharges"].sum()
    churn_revenue = segment[segment["Churn"] == "Yes"]["MonthlyCharges"].sum()
    
    risk_pct = (churn_revenue / total_revenue) * 100
    
    signal_results.append({
        "Signal": name,
        "Total_Revenue": total_revenue,
        "Churn_Revenue": churn_revenue,
        "Revenue_Risk_%": risk_pct
    })

import pandas as pd
signal_df = pd.DataFrame(signal_results)

signal_df.sort_values("Revenue_Risk_%", ascending=False)

,Signal,Total_Revenue,Churn_Revenue,Revenue_Risk_%
3,Tenure 0-6,81067.95,49896.10,61.548491
0,Month-to-month,257294.15,120847.10,46.968460
2,Electronic check,180345.00,84288.75,46.737503
4,Senior Citizen,91154.85,38419.60,42.147620
1,Fiber optic,283284.40,114300.05,40.348162


### Priority Intervention Matrix

In [20]:
# Normalize revenue scale for comparison
max_revenue = signal_df["Total_Revenue"].max()

signal_df["Revenue_Weight"] = signal_df["Total_Revenue"] / max_revenue

# Create Priority Score
signal_df["Priority_Score"] = (
    signal_df["Revenue_Risk_%"] * signal_df["Revenue_Weight"]
)

signal_df.sort_values("Priority_Score", ascending=False)

,Signal,Total_Revenue,Churn_Revenue,Revenue_Risk_%,Revenue_Weight,Priority_Score
0,Month-to-month,257294.15,120847.10,46.968460,0.908254,42.659285
1,Fiber optic,283284.40,114300.05,40.348162,1.000000,40.348162
2,Electronic check,180345.00,84288.75,46.737503,0.636622,29.754109
3,Tenure 0-6,81067.95,49896.10,61.548491,0.286172,17.613430
4,Senior Citizen,91154.85,38419.60,42.147620,0.321779,13.562201


### Revenue Recovery Simulation

In [21]:
# Get churn revenue for top 2 signals
top_signals = ["Month-to-month", "Fiber optic"]

recovery_results = []

for signal in top_signals:
    churn_revenue = signal_df[signal_df["Signal"] == signal]["Churn_Revenue"].values[0]
    
    # Assume 20% recovery
    recovered_revenue = churn_revenue * 0.20
    
    recovery_results.append({
        "Signal": signal,
        "Current_Churn_Revenue": churn_revenue,
        "Potential_Recovery_20%": recovered_revenue
    })

import pandas as pd
recovery_df = pd.DataFrame(recovery_results)

recovery_df

,Signal,Current_Churn_Revenue,Potential_Recovery_20%
0,Month-to-month,120847.10,24169.42
1,Fiber optic,114300.05,22860.01


### Churn Reduction Scenarios

In [22]:
signals_conditions = {
    "Month-to-month": df["Contract"] == "Month-to-month",
    "Fiber optic": df["InternetService"] == "Fiber optic",
    "Electronic check": df["PaymentMethod"] == "Electronic check",
    "Tenure 0-6": df["tenure"] <= 6,
    "Senior Citizen": df["SeniorCitizen"] == 1
}

signal_results = []

for name, condition in signals_conditions.items():
    segment = df[condition]
    
    total_revenue = segment["MonthlyCharges"].sum()
    churn_revenue = segment[segment["Churn"] == "Yes"]["MonthlyCharges"].sum()
    
    risk_pct = (churn_revenue / total_revenue) * 100
    
    signal_results.append({
        "Signal": name,
        "Total_Revenue": total_revenue,
        "Churn_Revenue": churn_revenue,
        "Revenue_Risk_%": risk_pct
    })

signal_df = pd.DataFrame(signal_results)
signal_df

,Signal,Total_Revenue,Churn_Revenue,Revenue_Risk_%
0,Month-to-month,257294.15,120847.10,46.968460
1,Fiber optic,283284.40,114300.05,40.348162
2,Electronic check,180345.00,84288.75,46.737503
3,Tenure 0-6,81067.95,49896.10,61.548491
4,Senior Citizen,91154.85,38419.60,42.147620


In [23]:
scenarios = [0.10, 0.20, 0.30, 0.40]

top_signals = ["Month-to-month", "Fiber optic"]

scenario_results = []

for signal in top_signals:
    churn_revenue = signal_df[signal_df["Signal"] == signal]["Churn_Revenue"].values[0]
    
    for s in scenarios:
        recovered = churn_revenue * s
        
        scenario_results.append({
            "Signal": signal,
            "Improvement_%": int(s * 100),
            "Recovered_Revenue": recovered
        })

import pandas as pd
scenario_df = pd.DataFrame(scenario_results)

scenario_df

,Signal,Improvement_%,Recovered_Revenue
0,Month-to-month,10,12084.710
1,Month-to-month,20,24169.420
2,Month-to-month,30,36254.130
3,Month-to-month,40,48338.840
4,Fiber optic,10,11430.005
5,Fiber optic,20,22860.010
6,Fiber optic,30,34290.015
7,Fiber optic,40,45720.020


### Combined Sensitivity Comparison

In [ ]:
combined_recovery = (
    scenario_df
    .groupby("Improvement_%")["Recovered_Revenue"]
    .sum()
    .reset_index()
)

# Annual recovery
combined_recovery["Annual_Recovery"] = (
    combined_recovery["Recovered_Revenue"] * 12
)

combined_recovery

,Improvement_%,Recovered_Revenue,Annual_Recovery
0,10,23514.715,282176.58
1,20,47029.430,564353.16
2,30,70544.145,846529.74
3,40,94058.860,1128706.32


In [25]:
combined_recovery["Incremental_Gain"] = (
    combined_recovery["Recovered_Revenue"].diff()
)

combined_recovery

,Improvement_%,Recovered_Revenue,Annual_Recovery,Incremental_Gain
0,10,23514.715,282176.58,NaN
1,20,47029.430,564353.16,23514.715
2,30,70544.145,846529.74,23514.715
3,40,94058.860,1128706.32,23514.715


### Break-Even Retention Cost Estimation

In [ ]:
target_customers = df[df["risk_segment"].isin(["High", "Medium"])]

num_target_customers = target_customers.shape[0]

num_target_customers

3730

In [27]:
annual_recovery_20 = 564353.16  # from previous table

break_even_per_customer = annual_recovery_20 / num_target_customers

break_even_per_customer

151.30111528150135

### Revenue KPI Snapshot Calculation

In [28]:
# 1️⃣ Total Monthly Revenue
total_revenue = df["MonthlyCharges"].sum()

# 2️⃣ Revenue at Risk %
revenue_at_risk = (
    df[df["Churn"] == "Yes"]["MonthlyCharges"].sum() / total_revenue
) * 100

# 3️⃣ High Risk Revenue Share
high_risk_revenue_share = (
    df[df["risk_segment"] == "High"]["MonthlyCharges"].sum() / total_revenue
) * 100

# 4️⃣ Month-to-month Revenue Share
month_to_month_share = (
    df[df["Contract"] == "Month-to-month"]["MonthlyCharges"].sum() / total_revenue
) * 100

# 5️⃣ Fiber Revenue Share
fiber_share = (
    df[df["InternetService"] == "Fiber optic"]["MonthlyCharges"].sum() / total_revenue
) * 100

total_revenue, revenue_at_risk, high_risk_revenue_share, month_to_month_share, fiber_share

(np.float64(456116.6),
 np.float64(30.503351555282137),
 np.float64(14.71844041633214),
 np.float64(56.40973163441103),
 np.float64(62.107890833177315))